# NLG

## Changes to aitextgen to have it run

~~~~{.python}
# aitextgen/aitextgen.py
# Temp fix
del train_params["show_progress_bar"]
trainer = pl.Trainer(**train_params)
trainer.fit(train_model)

# transformers/tokenization_gpt2.py
def convert_tokens_to_string(self, tokens):
    """ Converts a sequence of tokens (string) in a single string. """
    # Remove instances of None in tokens
    tokens = list(filter(lambda a: a is not None, tokens))
    text = "".join(tokens)
    text = bytearray([self.byte_decoder[c] for c in text]).decode("utf-8", errors=self.errors)
    return text
~~~~


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

## aitextgen

In [14]:
from aitextgen import aitextgen

# Without any parameters, aitextgen() will download, cache, and load the 124M GPT-2 "small" model
ai = aitextgen(to_gpu=True)

INFO:aitextgen:Loading gpt2 model from /aitextgen.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


In [15]:
print("========1=========")
ai.generate()

print("========2=========")
ai.generate(n=3, max_length=100)

print("========3=========")
ai.generate(n=3, prompt="I believe in unicorns because", max_length=100)
# ai.generate_to_file(n=10, prompt="I believe in unicorns because", max_length=100, temperature=1.2)

========1=========

The United States has been engaged in a war with Afghanistan since the end of the Cold War. But at least that's what the U.S. government believes.

In the first decade of the 21st century, the U.S. has been involved in more wars than any other country in the world. This includes wars in Iraq and Afghanistan, the Middle East, the Balkans, and East Africa.

But as the U.S. has been waging them, the only country in the world that has not been involved in one of them is Afghanistan. The Taliban is not the only force in the Middle East to be fighting the U.S.-led war on terrorism.

The Taliban is also the only country in the world that has not been involved in a war with the U.S. or its allies.

This is according to the U.S. official who heads the U.N. Security Council. He is the former head of the U.N. peacekeeping mission in Afghanistan.

The U.S. official said that all of the Afghan government forces in Afghanistan are in place and that the Taliban has been fighting f

In [22]:
# 10,000 max length may produce ~1000 words but they veer widely off topic
ai.generate(n=1, prompt="I believe in unicorns because", max_length=100000)

I believe in unicorns because unicorns are the best way to show off your personality and your strengths. But I also believe in unicorns because I love them. I love them because they're always fun to play with and have fun with. I love them because they're the perfect way to show people a certain kind of person. I don't care if they're very attractive or not. I love them because they're the coolest thing in the world. They'll be just as cool as they are on stage. They're the best things to do for your family. They're the best things to do for your friends. They're the best things to do for you. And I think that's what it means to be able to have a unicorn for friends and family and be happy with them.

What are your favorite items?

I mean, it's pretty cool to have a little unicorn for my birthday present. But I think it's really cool to have a little unicorn for my graduation. And the most awesome thing I'll ever have is a unicorn for my birthday present. I'll never know!

What are you

In [23]:
ai.generate(n=1, prompt="Mary loved John with all her heart, but the stresses of being in prison were causing an inevitable rift in their relationship.", max_length=100000)

Mary loved John with all her heart, but the stresses of being in prison were causing an inevitable rift in their relationship. She was afraid that no one would ever know how to love John, but she did not want anyone to know how to love John. John became more and more distant, afraid that John would not be with her with him, and at the same time, he was still John's friend.


In his first year in prison, John had a good time in school. He was a good student, but he was also very much in love with his sister. He was a good boy, and his sister would often tell him that he should learn to love. He was not the type of boy who would go to school and be bullied, but he was a good boy, and he loved his sister. The only thing that made his sister feel jealous of his school was that she was his best friend. He would often tell her something about his sister that her sister would never see. He was afraid that he would not be able to love her because he was afraid that he would never feel the same

In [24]:
# More repetitive, but safer, less creative output
ai.generate(n=1, prompt="Mary loved John with all her heart, but the stresses of being in prison were causing an inevitable rift in their relationship.", max_length=100, temperature=0.2)

Mary loved John with all her heart, but the stresses of being in prison were causing an inevitable rift in their relationship.

"I was in prison for a long time, and I was a very good person," she said. "I was very close to John, and I was very close to my family. I had a lot of friends, and I had a lot of friends who were very close to John. I was very close to my family. I was very close to my friends


In [2]:
# Finetune CPU model
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

# The name of the downloaded Shakespeare text for training
file_name = "data/input.txt"

# Train a custom BPE Tokenizer on the downloaded text
# This will save two files: aitextgen-vocab.json and aitextgen-merges.txt,
# which are needed to rebuild the tokenizer.
train_tokenizer(file_name)
vocab_file = "aitextgen-vocab.json"
merges_file = "aitextgen-merges.txt"

# GPT2ConfigCPU is a mini variant of GPT-2 optimized for CPU-training
# e.g. the # of input tokens here is 64 vs. 1024 for base GPT-2.
config = GPT2ConfigCPU()
#config = None

# Instantiate aitextgen using the created tokenizer and config
ai = aitextgen(vocab_file=vocab_file, merges_file=merges_file, config=config)

# You can build datasets for training by creating TokenDatasets,
# which automatically processes the dataset with the appropriate size.
data = TokenDataset(file_name, vocab_file=vocab_file, merges_file=merges_file, block_size=64)

# Train the model! It will save pytorch_model.bin periodically and after completion.
# On a 2016 MacBook Pro, this took ~25 minutes to run.
#ai.train(data, batch_size=16, num_steps=5000)
ai.train(data, batch_size=4, num_steps=5000)

# Generate text from it!
ai.generate(10, prompt="ROMEO:")

INFO:aitextgen.tokenizers:Saving aitextgen-vocab.json and aitextgen-merges.txt to the current directory. You will need both files to build the GPT2Tokenizer.
INFO:aitextgen:Constructing GPT-2 model from provided config.
INFO:aitextgen:Using a custom tokenizer.


INFO:aitextgen.TokenDataset:Encoding 40,000 sets of tokens from data/input.txt.
GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
;
And me to the this am I I


I I the.
QUEEN:



I? you thee?

I:
The that


But it.
My be is the to my a'll you.

That of.
And to?

2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.

And the daughter:
What, and the king,
To is the cion;
The one, I have not not I will.

But then,
And all a father!
And a be good you.


My that be not; I was's not will you's
3,000 steps reached: saving model to /trained_model
3,000 steps reached: generating sample texts.
 me,
His life, she was my lord! I have I be
Now, the lord,
I have not will to you? you have to our crown?

Ay, I that will do have have not be you is,


That not be his lord, I'll will it
4,000 steps reached: saving model to /trained_model
4,000 steps reached: generating sample texts.
:
And thou, thou, and your name,
And and I were my lord.

Why thou thou say you are this well,

First Murderer:

Saving latest checkpoint..
INFO:lightning:Saving latest checkpoint..
INFO:aitextgen:Saving trained model pytorch_model.bin to /trained_model


;
I have not no; or thou; and that will be an king:
I have I'll not be a man?


ANGELO:
I have do to the man?


JULIET:
I'll I will I had now we do a ccee.


ROMEO:

To give you, to not your lord;
I have a good that's blood of a a
Of a king and all the honour was no; and make
And, sir, nor him,
And I are so, by her, and all,
And to your lord: for
ROMEO:
To the world with a noble lord,
And in our way and with my lord,
To give thee, we have a very duke, my mother,
Or it well but with them; in the rest,
My own husband, that she's brother, that I have the blood
I
ROMEO:
I have I had not so well:
Thou, not my name of this'st is a very.

GLOUCESTER:

GLOUCESTER:
I have
The brother?


GLOUCESTER:
DUKE OF YORK:
I have thy hands.

First Citizen:
BENVOLIO:
ROMEO:

G ANNE:
That'st the lord, my lord, sir.


DUKE VINCENTIO:
Well, I do you must not.

DUKE VINCENTIO:

CORIOLANUS:
I'll have a tresed be:

He is you, I make it?

ROMEO:
I'll, in me, and I do.

BRUTUS:
I do thee,

GLOUCESTER:
What, I am, 

In [6]:
# Load CPU Shakespeare model
ai = aitextgen(model="small_models/cpu_shakespeare/pytorch_model.bin", config="small_models/cpu_shakespeare/config.json",
               vocab_file="small_models/cpu_shakespeare/aitextgen-vocab.json", merges_file="small_models/cpu_shakespeare/aitextgen-merges.txt")
ai.generate(10, prompt="ROMEO:")

INFO:aitextgen:Loading GPT-2 model from provided models/cpu_shakespeare/pytorch_model.bin.
INFO:aitextgen:Using a custom tokenizer.


ROMEO:
What, my lord, sir, but you should to you.

KING RICHARD III:
Which, he shall you, as you'll be a man is at.

PETRUCHIO:
No; you, sir, which I'll not come;

KING RICHARD III:
And
ROMEO:
Titer the own good honour of him have a
Of the brother's all his brother's
We have the father is the life'st-morrow's death;
I know the earth of this I make this.

POLIXENES:
That is, sir's lord, sir:

ROMEO:

KATHARINA:
Come, that, I mean, my lord, what's, that you,
And, and that what is he is an lord,
To am a motepoted, in the death,
I shall not have I have
To do all that can see
ROMEO:
I have not.

Clown:


DUKE VINCENTIO:
Not I
To do you, which I pray you?

LUCENTIO:
I would you, sir, sir, let me,, he shall
I may be a king'st's that he not,
The
ROMEO:


CORIOLANUS:
I can have I, I am
I'll be my lord; it.

BUCKINGHAM:
No, we'll hear you.

LUCIO:

JULIET:
To be the mind, as you, but your king and are;
That is a house
ROMEO:
First VINCENTIO:
The brother is.

MENENIUS:
I am the matter?

LEONTES:


In [1]:
# Finetune GPU GP2 model
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

# The name of the downloaded Shakespeare text for training
file_name = "data/input.txt"

# Train a custom BPE Tokenizer on the downloaded text
# This will save two files: aitextgen-vocab.json and aitextgen-merges.txt,
# which are needed to rebuild the tokenizer.
train_tokenizer(file_name)
vocab_file = "aitextgen-vocab.json"
merges_file = "aitextgen-merges.txt"

# GPT2ConfigCPU is a mini variant of GPT-2 optimized for CPU-training
# e.g. the # of input tokens here is 64 vs. 1024 for base GPT-2.
# config = GPT2ConfigCPU()
config = None

# Instantiate aitextgen using the created tokenizer and config
ai = aitextgen(vocab_file=vocab_file, merges_file=merges_file, config=config)

# You can build datasets for training by creating TokenDatasets,
# which automatically processes the dataset with the appropriate size.
data = TokenDataset(file_name, vocab_file=vocab_file, merges_file=merges_file, block_size=64)

# Train the model! It will save pytorch_model.bin periodically and after completion.
# On a 2016 MacBook Pro, this took ~25 minutes to run.
#ai.train(data, batch_size=16, num_steps=5000)
ai.train(data, batch_size=4, num_steps=5000)

# Generate text from it!
ai.generate(10, prompt="ROMEO:")

INFO:aitextgen.tokenizers:Saving aitextgen-vocab.json and aitextgen-merges.txt to the current directory. You will need both files to build the GPT2Tokenizer.
INFO:aitextgen:Loading gpt2 model from /aitextgen.
INFO:aitextgen:Using a custom tokenizer.


INFO:aitextgen.TokenDataset:Encoding 40,000 sets of tokens from data/input.txt.
GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
.

CORIOLANUS:
This is the moon of my cousin?

CORIOLANUS:
I am a word more, let them go.

KING HENRY VI:
This is this, come to our v-morrow.

CORIOLANUS:
This is this, our mother; let them be gone, let them be gone.

CORIOLANUS:
I am a king, there's name'st;
And they the other that the poor 'tis for the earth,
Which, and, how the fverel is a man's life; or so were them,
Is, how he in this noble eyes:
Who, in him to the queen, the poor it be gone, they shall be gone,
This is all the mind: yet, the end of my heart's name:
As I come thy hand?


CORIOLANUS:
O, my lord.

But, for thee; whose you
That I hear and the queen?

CORIOLANUS:
My command, and you shall be gone.

CORIOLANUS:
We are too, so he me; how he is this good's mind;
For I cannot she

CORIOLANUS:
Than all me a noble pn:
2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.
:
Where, is t

Saving latest checkpoint..
INFO:lightning:Saving latest checkpoint..


,
And let him be so much. If he be
To speak of his friends, which he were his wife
The Volsces are he be so much, which he were it
of he please our countrymen.

First Gentleman:
No, but by his countrymen.

Second Gentleman:
Go, go, give me thy hand:
He cannot be his countrymen.

First Gentleman:
Not he not, my lord; he is gone,

And therefore then the time to the king's; and his countrymen.

As he is gone.


First Gentleman:
You do not the heavens.


LUCIO:



First Gentleman:



First Gentleman:
Your true; he is gone.



LUCIO:




LUCIO:

That's he is gone, sir,
If he be the heavens.


Go then the best I have made him here he have his countrymen.
The man; and he is gone.

Second Gentleman:

TRANIO:
You, he be gone, sir.



LUCIO:


SICINIUS:
And why have it be


INFO:aitextgen:Saving trained model pytorch_model.bin to /trained_model


ROMEO:
I am the best of my father's death;
And I'll be, I am the best of my heart.

PETRUCHIO:
We have the best of my heart.

LUCENTIO:
I will not know it, I have confessor;
And then I have not confessor;
I'll have you know it not that's my heart.

BIONDELLO:
We have you shoulder: be gone, my heart; I am not, sweet wife;
And then I will be too, or the king not.


PETRUCHIO:
A hundred then, for you know you well.


LUCENTIO:

BIONDELLO:
You thank you, sir, sir, my lord; 'tis he shoulder a good.


BIONDELLO:

BIONDELLO:
And so.


PETRUCHIO:
PETRUCHIO:
I had you, sir.

' the best?

BIONDELLO:

PETRUCHIO:
And so.

PETRUCHIO:

BIONDELLO:

The best of your part;

The best.
The best of your heart is it is.

No.

BIONDELLO:

BIONDELLO:

As your father is my
ROMEO:
As for the people's son, or he is dead already,
To be a wild-speaky, or I will not carry.

FRIAR LAURENCE:
I have a wild-speaky and he is dead;
And will be pasty true and in the morning.
Take up your own report, and by his own report

In [3]:
# Load GPU GPT2 Shakespeare model
ai = aitextgen(model="large_models/gpu_gpt2_shakespeare/pytorch_model.bin", config="large_models/gpu_gpt2_shakespeare/config.json",
               vocab_file="large_models/gpu_gpt2_shakespeare/aitextgen-vocab.json", merges_file="large_models/gpu_gpt2_shakespeare/aitextgen-merges.txt",
               to_gpu=True)
ai.generate(10, prompt="ROMEO:")

INFO:aitextgen:Loading GPT-2 model from provided large_models/gpu_gpt2_shakespeare/pytorch_model.bin.
INFO:aitextgen:Using a custom tokenizer.


ROMEO:
No, no, no; no, no, no, is it not
The other for the bagozy, as he does,
As he hath not, my lord, and so much a
Some sow't; but so I'll not be
In the volume of the volume of the fretting,
And yet in his own report of the duke's daughter,
That I must not believe in the state,
The loving friends he is well as he is something: if he comes
To be of his own report, if he of him.


Lord:
O, he's the worst, he of his own report!
O, he is the crown.


Lord:
Is he, he not so, he's the crown to him.



Lord:

AThey will be.

Is he not so.


Lord:
And he's the crown.


AThey are he may he of a great:
And he is the crown, he is it is he of his name, he is he is his injury.

I will be the crown.


TRANIO:

ROMEO:
He has a little gall'd in the people,
That, being a little gall'd in the people,
With this your own tears; and, in the people,
That never saw it, and, in the people,
O, that it were so, he hath made a bonce.

First Huntsman:
He is the people, he that have not so we were the people.

